# SECTION 0.0: COLAB PREP

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
# %cd gdrive/'My Drive'/hddmnn_tutorial

# !pip uninstall tensorflow  # tf 2.3.0 is loaded by default --> very slow on our networks
# !pip install tensorflow-gpu==1.15 # tf 1.15 ~ 4 times faster (feel free to test this out)
# !pip install pymc
# !pip install kabuki
# !pip install -U --no-deps git+https://github.com/alexanderfengler/hddm@nn_likelihood

[Errno 2] No such file or directory: 'gdrive/My Drive/hddmnn_tutorial'
/Users/afengler/OneDrive/git_repos/hddmnn_tutorial
Found existing installation: tensorflow 1.15.0
Uninstalling tensorflow-1.15.0:
  Would remove:
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/estimator_ckpt_converter
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/freeze_graph
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/saved_model_cli
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/tensorboard
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/tf_upgrade_v2
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/tflite_convert
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/toco
    /Users/afengler/opt/miniconda3/envs/hddm_dev/bin/toco_from_protos
    /Users/afengler/opt/miniconda3/envs/hddm_dev/lib/python3.7/site-packages/tensorflow-1.15.0.dist-info/*
    /Users/afengler/opt/miniconda3/envs/hddm_dev/lib/python3.7/site-packages/tensorflow/*
    /Users/afengler/opt/miniconda3/envs/hddm_

In [ ]:
# MODULE IMPORTS ----

# 
import hddm

# Make simulators visible
import sys
sys.path.append('simulators')

# Data management
import pandas as pd
import numpy as np
import pickle

# Plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Stats functionality
from statsmodels.distributions.empirical_distribution import ECDF

# CUSTOM IMPORTS (LOCAL FILES) --------
import cddm_data_simulation as cds
import boundary_functions as bf

from helper_functions import simulator
from helper_functions import simulator_covariate
from helper_functions import simulator_stimcoding
from helper_functions import model_plot
from helper_functions import caterpillar_plot
from helper_functions import posterior_pair_plot
# from helper_functions import hddm_preprocess_hierarchical
from helper_functions import simulator_condition_effects
from helper_functions import make_parameter_sets
from helper_functions import _make_trace_plotready_condition
from helper_functions import hddm_preprocess
from helper_functions import simulator_hierarchical
from helper_functions import posterior_predictive_plot
# --------------------------------------

In [ ]:
p_outlier = []
include_conf = {'angle': ['z', 'theta'] + p_outlier,
                'weibull_cdf':['z', 'alpha', 'beta'] + p_outlier,
                'full_ddm': ['z', 'st', 'sv', 'sz'] + p_outlier,
                'levy': ['z', 'alpha'] + p_outlier,
                'ornstein': ['z', 'g'] + p_outlier,
                'ddm_sdv': ['z', 'sv'] + p_outlier,
                'ddm': ['z']}

# Single Subjects

In [ ]:
model = 'ornstein'
n_samples = 10000
p_outlier = [] # ['p_outlier']


params = make_parameter_sets(model = model,
                             n_parameter_sets = 1)

simulations = simulator(theta = params.values[0],
                        model = model,
                        n_samples = n_samples)

hddm_data_single_subject = hddm_preprocess(simulations)



# hddm_model_ddm_analytic_single_subject = hddm.HDDMnn(hddm_data_ddm_single_subject, 
#                                                          model = 'ddm_analytic',
#                                                          informative = False,
#                                                          include = ['z'],
#                                                          is_group_model = False)

# Defining a hddm model
hddm_model_single_subject = hddm.HDDMnn(hddm_data_single_subject,
                                        model = model,
                                        informative = False,
                                        include = include_conf[model],
                                        p_outlier = 0.0,
                                        w_outlier = 0.01,
                                        is_group_model = False)

In [ ]:
nmcmc = 300
hddm_model_single_subject.sample(nmcmc, 
                                     burn = 100)

In [ ]:
print('dic: ', hddm_model_single_subject.dic)
print(hddm_model_single_subject.gen_stats()['mean'])
print('gt params: ', params)

In [ ]:
params

In [ ]:
posterior_predictive_plot(posterior_samples = hddm_model_single_subject.get_traces(), #hddm_model_weibull_single_subject.get_traces(),
                          ground_truths_parameters = params.values[0], #weibull_params.values[0],
                          ground_truths_data = np.concatenate([simulations[0], simulations[1]], axis = 1), # None
                          n_plots = 1,
                          cols = 1,
                          model_fitted = model,
                          model_gt = None, #'levy', #'weibull_cdf',
                          datatype = 'single_subject',
                          n_post_params = 200,
                          samples_by_param = 100,
                          xlimit = 10,
                          bin_size = 0.025,
                          hist_linewidth = 1)

In [ ]:
# Caterpillar Plot: (Parameters recovered ok?)
caterpillar_plot(posterior_samples = hddm_model_single_subject.get_traces(),
                 ground_truths = params.values[0], #weibull_params.values[0],
                 model = model,
                 datatype = 'single_subject',
                 drop_sd = True)

# CONDITION

In [ ]:
model = 'ornstein'
n_samples_by_condition = 500
p_outlier = [] # ['p_outlier']


hddm_data_condition, gt_condition, gt_mat_condition = simulator_condition_effects(n_conditions = 4,
                                                                                  n_samples_by_condition = n_samples_by_condition,
                                                                                  condition_effect_on_param = ['v', 'a'],
                                                                                  model = model)


hddm_model_condition = hddm.HDDMnn(hddm_data_condition, 
                                   model = model,
                                   informative = False,
                                   include = include_conf[model],
                                   p_outlier = 0.,
                                   w_outlier = 0.,
                                   is_group_model = False, 
                                   depends_on = {'v': 'condition', 'a': 'condition'})


hddm_model_condition.sample(500, burn = 100)

In [ ]:
# Caterpillar Plot: (Parameters recovered ok?)
caterpillar_plot(posterior_samples = hddm_model_condition.get_traces(),
                 ground_truths = gt_condition, #weibull_params.values[0],
                 model = model,
                 datatype = 'condition',
                 drop_sd = True)

# GROUP

In [2]:
model = 'angle'
n_subjects = 10
n_samples_by_subject = 500
p_outlier = [] # ['p_outlier']


# Simulate data angle
hddm_data_hierarchical, gt_hierarchical, gt_mat_hierarchical = simulator_hierarchical(n_subjects = n_subjects,
                                                                                      model = model,
                                                                                      n_samples_by_subject = n_samples_by_subject)


hddm_hierarchical_model = hddm.HDDMnn(hddm_data_hierarchical, 
                                                    model = model,
                                                    informative = False,
                                                    include = include_conf[model],
                                                    p_outlier = 0,
                                                    w_outlier = 0,
                                                    is_group_model = True)

# # Sampling:
nmcmc = 500
hddm_hierarchical_model.sample(nmcmc,
                               burn = 100)

NameError: name 'simulator_hierarchical' is not defined

In [ ]:
# Subset data?
hddm_data_hierarchical_limited = hddm_data_hierarchical.iloc[:2500]
str_allowed = ['005', 
               '006', 
               '007', 
               '008', 
               '009', 
               '010', 
               '011', 
               '012', 
               '013', 
               '014', 
               '015', 
               '016',
               '017',
               '018',
               '019']
gt_hierarchical_limited = (gt_hierarchical).copy()
for str_tmp in gt_hierarchical.keys():
    for str_allowed_tmp in str_allowed:
        if str_allowed_tmp in str_tmp:
            gt_hierarchical_limited.pop(str_tmp)

In [ ]:
# Caterpillar Plot
caterpillar_plot(posterior_samples = hddm_hierarchical_model.get_traces(),
                 ground_truths = gt_hierarchical,
                 model = model,
                 datatype = 'hierarchical',
                 x_lims = [-2, 2],
                 aspect_ratio = 2,
                 drop_sd = False,
                 save = False)

# Regression

In [ ]:
# Ornstein params
model = 'ddm'
n_samples = 1000

params =  make_parameter_sets(model = model,
                              n_parameter_sets = 1)

simulations = simulator(theta = params.values[0],
                        model = model, 
                        n_samples = n_samples)

hddm_data_single_subject_regress = hddm_preprocess(simulations)

# Add a regressor:
hddm_data_single_subject_regress['BOLD'] = np.random.uniform(low = - 1, high = 1, size = n_samples)

#v_reg = {'model': 'v ~ 1 + BOLD', 'link_func': lambda x: x}
#a_reg = {'model': 'a ~ 1 + BOLD', 'link_func': lambda x: x}
#t_reg = {'model': 't ~ 1 + BOLD', 'link_func': lambda x: x}
g_reg = {'model': 'a ~ 1 + BOLD', 'link_func': lambda x: x}

reg_descr = [g_reg] #[v_reg, a_reg, t_reg, g_reg] # theta_reg]

hddm_reg = hddm.HDDMnnRegressor(hddm_data_single_subject_regress, 
                                reg_descr, 
                                include = set(include_conf[model]),
                                model = model)

In [ ]:
# # Sampling:
nmcmc = 500
hddm_reg.sample(nmcmc,
                burn = 100)